# Tutorial on pre-training of Chinese-LLaMA-7B

More info: https://github.com/ymcui/Chinese-LLaMA-Alpaca

## Install Dependencies

In [3]:
!pip install transformers==4.28.1
!pip install git+https://github.com/huggingface/peft.git@13e53fc
!pip install datasets
!pip install sentencepiece
!pip install deepspeed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git (to revision 13e53fc) to /tmp/pip-req-build-6fdyuocz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-6fdyuocz
  Running command git checkout -q 13e53fc
  Resolved https://github.com/huggingface/peft.git to commit 13e53fc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 18.8 MB/s eta 0:00:00
  Created

## Clone our repository





In [4]:
!git clone https://github.com/ymcui/Chinese-LLaMA-Alpaca.git

Cloning into 'Chinese-LLaMA-Alpaca'...
remote: Enumerating objects: 911, done.
remote: Counting objects: 100% (352/352), done.
remote: Compressing objects: 100% (233/233), done.
remote: Total 911 (delta 135), reused 211 (delta 117), pack-reused 559
Receiving objects: 100% (911/911), 18.13 MiB | 10.57 MiB/s, done.
Resolving deltas: 100% (527/527), done.


## Pre-training for LLaMA-7B

This follows the setting in https://github.com/ymcui/Chinese-LLaMA-Alpaca/wiki/Pretraining-Script, except that to simplify the tutorial,
- only train 100 steps
- use a sample data file built from alpaca_data_zh_51k.json

In [ ]:
!mkdir Chinese-LLaMA-Alpaca/pt_data
!cp Chinese-LLaMA-Alpaca/data/pt_sample_data.txt Chinese-LLaMA-Alpaca/pt_data

In [9]:
!cd Chinese-LLaMA-Alpaca/scripts && torchrun --nnodes 1 --nproc_per_node 1 run_clm_pt_with_peft.py \
    --deepspeed ds_zero2_no_offload.json \
    --model_name_or_path decapoda-research/llama-7b-hf \
    --tokenizer_name_or_path ziqingyang/chinese-llama-lora-7b \
    --dataset_dir /content/Chinese-LLaMA-Alpaca/pt_data \
    --data_cache_dir data_cache \
    --validation_split_percentage 0.001 \
    --per_device_train_batch_size 1 \
    --do_train \
    --fp16 \
    --seed $RANDOM \
    --max_steps 100 \
    --lr_scheduler_type cosine \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --save_strategy steps \
    --save_total_limit 3 \
    --save_steps 50 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 8 \
    --block_size 512 \
    --output_dir /content/output_model \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --lora_rank 8 \
    --lora_alpha 32\
    --trainable q_proj,v_proj,k_proj,o_proj,gate_proj,down_proj,up_proj \
    --modules_to_save embed_tokens,lm_head \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --gradient_checkpointing \
    --ddp_find_unused_parameters False

2023-05-12 06:15:17.883035: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-05-12 06:15:18,834] [INFO] [comm.py:622:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
05/12/2023 06:15:21 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True
[INFO|configuration_utils.py:668] 2023-05-12 06:15:21,697 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--decapoda-research--llama-7b-hf/snapshots/5f98eefcc80e437ef68d457ad7bf167c2c6a1348/config.json
[INFO|configuration_utils.py:720] 2023-05-12 06:15:21,698 >> Model config LlamaConfig {
  "_name_or_path": "decapoda-research/llama-7b-hf",
  "architectures": [
    "LLaMAForCausalLM"
  ],
  "bos_token_id": 0,
  "eos_token_id": 1,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "

After training, rename saved `pytorch_model.bin` to `adapter_model.bin`

In [10]:
!mkdir output_model/peft_model
!mv output_model/pytorch_model.bin output_model/peft_model/adapter_model.bin

Lastly, you need to manually create an `adapter_config.json` under `peft_model` and fill in the hyperparamters such as `lora_rank`, `lora_alpha` etc., whose content and 
format can be referenced from the corresponding file in Chinese-LLaMA-LoRA.